In [3]:
from collections import namedtuple
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dipy.core.gradients import gradient_table
from dipy.data import get_fnames, default_sphere
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti

In [4]:
InputData = namedtuple("InputData", ("data", "affine", "gtab", "labels", "folder"))


def load_data(folder: Path, ident: str, description: pd.DataFrame) -> InputData:
    hardi_fname, hardi_bval_fname, hardi_bvec_fname = [str(folder / (ident + end)) for end in [".nii.gz", ".bval", ".bvec"]]
    data, affine = load_nifti(hardi_fname)

    bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
    gtab = gradient_table(bvals, bvecs)
    
    labels = description[description.Subject == ident]
    return InputData(data, affine, gtab, labels, folder)


In [5]:
labels = pd.read_excel(r"F:\Uni\master\teamproject\raw_Data\Data specification.xlsx")
labels

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format
0,298129,094_S_4282,AD,M,90,25,DTI,Axial DTI,Original,4/17/2012,DCM
1,376313,052_S_5062,AD,F,71,24,DTI,Axial DTI,Original,2013-12-06 00:00:00,DCM
2,1184057,022_S_6013,AD,F,63,103,DTI,Axial DTI,Original,6/21/2019,DCM
3,380961,027_S_4802,AD,M,84,28,DTI,Axial DTI,Original,7/16/2013,DCM
4,1186912,129_S_6763,AD,F,82,1,DTI,Axial DTI,Original,7/15/2019,DCM
5,250176,003_S_4136,AD,M,67,22,DTI,Axial DTI,Original,2011-10-08 00:00:00,DCM
6,254902,003_S_4142,AD,F,90,22,DTI,Axial DTI,Original,8/31/2011,DCM
7,261479,094_S_4089,AD,M,74,24,DTI,Axial DTI,Original,10/18/2011,DCM
8,1161685,029_S_6289,CN,F,58,102,DTI,Axial DTI,Original,4/30/2019,DCM
9,829304,002_S_1280,CN,F,81,101,DTI,Axial DTI,Original,3/13/2017,DCM


In [11]:
CSD_Peaks = namedtuple("CSD_Peaks", ("peak_model_file", "label"))

def peaks_from_data(input_data: InputData):
    from dipy.data import default_sphere
    from dipy.reconst.csdeconv import auto_response, ConstrainedSphericalDeconvModel
    from dipy.direction import peaks_from_model
    from dipy.io.peaks import save_peaks, save_nifti
    from datetime import datetime
    
    print(f"{datetime.now()}: Fitting CSD Model for subject {input_data.labels.Subject}")
    
    response, ratio = auto_response(input_data.gtab, input_data.data, roi_radius = 10, fa_thr=0.7)
    
    csd_model = ConstrainedSphericalDeconvModel(input_data.gtab, response)

    pam =  peaks_from_model(model=csd_model,
                            data=input_data.data,
                            sphere=default_sphere,
                           relative_peak_threshold=0.5,
                           min_separation_angle=25,
                           parallel=True)
    
    filename = str(input_data.folder / (input_data.labels.Subject[0]))
    
    save_peaks(filename + ".h5", pam)
    save_nifti(filename + ".nii.gz", pam)
    
    print(f"saved sucessfully: {filename}")
    return CSD_Peaks(filename, input_data.label)

In [12]:
def load_data_subjects(folder: Path, labels: pd.DataFrame):
    def wrapper(subject: str):
        return load_data(folder, subject, labels)
    return wrapper

In [13]:
subjects_to_data = load_data_subjects(Path(r"F:\Uni\master\teamproject\preprocessed"), labels)
pipeline = map(peaks_from_data, map(subjects_to_data, labels.Subject))

In [ ]:
processed = list(pipeline)

2020-07-03 16:25:58.223970: Fitting CSD Model for subject 0    094_S_4282
Name: Subject, dtype: object


C:\tools\miniconda3\lib\site-packages\dipy\reconst\csdeconv.py:132: UserWarning: Number of parameters required for the fit are more than the actual data points
  warnings.warn(msg, UserWarning)
